In [2]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import random
from zipfile import ZipFile
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from skimage.io import imread, imsave
from skimage import color, exposure
from skimage.transform import rescale
from skimage.util import view_as_blocks
from multiprocessing import Pool
from skimage.color import rgb2gray
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import multiprocess as mp
import pickle
import tensorflow as tf
from multiprocessingNotebook import runt
"""
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed """
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TensorFlow and tf.keras
import tensorflow as tf
from keras import callbacks

PICKLE_PATH = 'test_data.pickle'

NUM_TRAINING_IMAGES = 5
NUM_TESTING_IMAGES = 5

NUM_SPACES_PER_BOARD = 2
PICTURE_DIMENSIONS = 400
block_width = PICTURE_DIMENSIONS // 8
block_height = PICTURE_DIMENSIONS // 8
num_channels = 3

In [3]:
from zipfile import ZipFile

def getFilePaths(filePath):
    # opens the chess boards.zip file into the chessBoards object
    chessBoards = ZipFile(filePath, 'r')

    test = []
    train = []

    # extract the file names from the train and test folders in the zip archive
    for file in chessBoards.namelist():
        # fills the file names from within the dataset subfolder
        if file[:4] == 'test':
            test.append(file)
        if file[:5] == 'train':
            train.append(file)

    return train, test

zipFilePath = "../chess boards.zip"

trainPaths, testPaths = getFilePaths(zipFilePath)
print("Files in train: ", len(trainPaths))
print("Files in test: ", len(testPaths))

Files in train:  80000
Files in test:  20000


In [4]:
# Parallel Loads Board Images for Training Purposes
def HogTransform(img):
    first_image_gray = color.rgb2gray(img)

    fd, hog_image = hog(
        first_image_gray,
        orientations=8,
        pixels_per_cell=(8, 8),
        cells_per_block=(1, 1),
        visualize=True,
        block_norm='L2-Hys',
        feature_vector=True
    )

    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    hog_image_uint8 = (hog_image_rescaled * 255).astype(np.uint8)
    return hog_image_uint8

def read_and_store_modified_chess_images():
    return

def fen_from_position(position, fen_string):
    rows = position // 8
    cols = position % 8

    character = fen_string[rows * 8 + cols]

    if character.isdigit():
        return ' ' * int(character)
    else:
        return character

def fen_from_filename(filename):
    parts = filename.split('/')
    fen_part = parts[-1].split('.')[0]

    fen_string = ''.join([' ' * int(char) if char.isdigit() else char for char in fen_part])
    fen_string = fen_string.replace('-', '')

    return fen_string

src = zipFilePath
random.shuffle(trainPaths)

print("Running...")
lock = mp.Lock()
manager = mp.Manager()
processorCores = 12
#data_dict = manager.dict({'fenstring': [], 'data': []})
data_dict = manager.list()
with mp.Pool(processes=processorCores) as p:
    args = [(fileName, src, fen_from_filename, fen_from_position, NUM_SPACES_PER_BOARD, data_dict) for fileName in trainPaths[:NUM_TRAINING_IMAGES]]
    results = [0]
    results = p.starmap(runt, args)

# Access the shared dictionary
print(len(data_dict))

combined_results = {'fenstring': [], 'data': []}

# Initialize lists to store all data and fenstring items
all_data_items = []
all_fenstring_items = []

for result in data_dict:
    # Extend all_data_items with result['data']
    all_data_items.extend(result['data'])

    # Extend all_fenstring_items with result['fenstring']
    all_fenstring_items.extend(result['fenstring'])

# Assign the combined lists to combined_results
combined_results['data'] = all_data_items
combined_results['fenstring'] = all_fenstring_items

joblib.dump(combined_results, PICKLE_PATH)
read_and_store_modified_chess_images()

Running...
19


In [6]:
# extracts the uniques spaces from a chess board images and saves them in directories of like squares
def boardToSpaces():

    # finds every unique square/piece combination in the fen string
    def getUniqueImageLocations(filename):
        #fenString = fen_from_filename("1b1B1b2-2pK2q1-4p1rB-7k-8-8-3B4-3rb3.jpeg")
        fenString = fen_from_filename(filename)

        # turn the fen string into arrays in the board shape
        stringPerRow = []
        for row in range(8):
            rowBeggining = row * 8
            rowEnding = row * 8 + 8
            stringPerRow.append(fenString[rowBeggining:rowEnding])

        uniqueImages = {
            # empty light and dark squares
            " l": False,
            " d": False,
            # white pieces on dark square
            "WPd": False,
            "WRd": False,
            "WNd": False,
            "WBd": False,
            "WQd": False,
            "WKd": False,
            # white pieces on light square
            "WPl": False,
            "WRl": False,
            "WNl": False,
            "WBl": False,
            "WQl": False,
            "WKl": False,
            # black pieces on dark square
            "Bpd": False,
            "Brd": False,
            "Bnd": False,
            "Bbd": False,
            "Bqd": False,
            "Bkd": False,
            # black pieces on light square
            "Bpl": False,
            "Brl": False,
            "Bnl": False,
            "Bbl": False,
            "Bql": False,
            "Bkl": False
        }

        # fills location of unique combinations to uniqueImages
        for row in range(8):
            for column in range(8):
                contents = spaces[stringPerRow[row][column]] + spaceColor[row][column]
                if not uniqueImages[contents]:
                    #print("adding ", row, ", ", column, "to ", contents)
                    uniqueImages[contents] = str(row) + str(column)

        return uniqueImages

    inputFilePath = "../chess boards/"
    outputFilePathBase = "../chess boards/individualTest/"

    fileNameCounter = {
        "Bp": 0,
        "WP": 0,
        "Br": 0,
        "WR": 0,
        "Bn": 0,
        "WN": 0,
        "Bb": 0,
        "WB": 0,
        "Bq": 0,
        "WQ": 0,
        "Bk": 0,
        "WK": 0,
        " ": 0
    }
    spaceColor = ["ldldldld", "dldldldl", "ldldldld", "dldldldl", "ldldldld", "dldldldl", "ldldldld", "dldldldl"]
    spaces = {
        "P": "WP",
        "p": "Bp",
        "R": "WR",
        "r": "Br",
        "N": "WN",
        "n": "Bn",
        "B": "WB",
        "b": "Bb",
        "Q": "WQ",
        "q": "Bq",
        "K": "WK",
        "k": "Bk",
        " ": " "
        }

    filesRead = 0
    for inputFileName in testPaths:

        print("on file:", filesRead)
        filesRead += 1
        #inputFileName = trainPaths[index]
        uniqueLocations = getUniqueImageLocations(inputFileName[5:])
        image = imread(os.path.join(inputFilePath, inputFileName))
        #scaledImage = rescale(image, .5, channel_axis = -1)
        #grayImage = color.rgb2gray(scaledImage)
        grayImage = color.rgb2gray(image)
        scaledGray = rescale(grayImage, .5)
        normalizedGray = (scaledGray * 255).astype(np.uint8)

        #print("Input filename:", inputFileName)

        for key in uniqueLocations:
            if uniqueLocations[key]:
                if key[0] == " ":
                    outputFilePath = outputFilePathBase + "e" + "/"
                    outputFileName = "e" + "_" + str(fileNameCounter[key[0]]) + ".jpeg"
                    fileNameCounter[key[0]] += 1
                else:
                    outputFilePath = outputFilePathBase + key[:2] + "/"
                    outputFileName = key[:2] + "_" + str(fileNameCounter[key[:2]]) + ".jpeg"
                    fileNameCounter[key[:2]] += 1

                #print("outputFileName: ", outputFileName)
                yLoc = 25 * int(uniqueLocations[key][0])
                xLoc = 25 * int(uniqueLocations[key][1])
                outputImage = normalizedGray[yLoc : yLoc + 25, xLoc : xLoc + 25]
                imsave(os.path.join(outputFilePath, outputFileName), outputImage, check_contrast=False)

convertBoards = False
if convertBoards:
    boardToSpaces()


In [11]:
import os
import pathlib
"""
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed """
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TensorFlow and tf.keras
import tensorflow as tf
from keras import callbacks

# wsl file path
#dataset_url = "file:///mnt/c/Users/kevdi/Desktop/UA Files/Applied Machine Learning/Project 3/chess boards.zip"
datasetDirectoryPath = "../chess boards/individualTrain"
classname = ["BB",
"BK",
"BN",
"BP",
"BQ",
"BR",
"e",
"WB",
"WK",
"WN",
"WP",
"WQ",
"WR"]

try:
    train_images_ds = tf.data.Dataset.load('./dataSets/traindata')
    print("Train Dataset loaded.\n")
except:
    print("Creating training dataset...")
    train_images_ds = tf.keras.utils.image_dataset_from_directory(
        datasetDirectoryPath,
        seed = 1337,
        image_size = (25, 25), # resizes images to specified size
        shuffle = True, # sorted alphanumerically otherwise
        batch_size=32,
        label_mode = "categorical",
        labels = "inferred",
        class_names = classname,
        color_mode = "grayscale",
    )
    print("Training dataset Created.")

    print("Saving training dataset...")
    tf.data.Dataset.save(train_images_ds, './dataSets/traindata')
    print("Training dataset saved.")

try:
    test_images_ds = tf.data.Dataset.load('./dataSets/testdata')
    print("Test Dataset loaded.\n")
except:
    print("Creating testing dataset...")
    train_images_ds = tf.keras.utils.image_dataset_from_directory(
        datasetDirectoryPath,
        seed = 1337,
        image_size = (25, 25), # resizes images to specified size
        shuffle = True, # sorted alphanumerically otherwise
        batch_size=32,
        label_mode = "categorical",
        labels = "inferred",
        class_names = classname,
        color_mode = "grayscale",
    )
    print("Testing dataset Created.")

    print("Saving Testing dataset...")
    tf.data.Dataset.save(train_images_ds, './dataSets/testdata')
    print("Testing dataset saved.")

def get_weight_dict(dir):
    most_samples=0
    class_weight={}
    class_list=os.listdir(dir) # dir is the directory with the training samples organized by class
    for c in (class_list): # iterate through class directories, find number of samples in each class then find class with highest number of samples
        c_path=os.path.join(dir,c)
        if os.path.isdir(c_path):
            length=len(os.listdir(c_path)) # determine number of samples in the class directory
            if length>most_samples:
                most_samples=length
    for i,c in enumerate(class_list): #iterate through class directories, find number of samples in each and divide total_samples by length
        c_path=os.path.join(dir,c)
        if os.path.isdir(c_path):
            length=len(os.listdir(c_path)) # number of samples inclass directory
            class_weight[i]=most_samples/length
            #print (i,most_samples, class_weight[i])
    return class_weight
classWeights = get_weight_dict("../chess boards/individualTest")
#print("classWeights:", classWeights)

Creating training dataset...
Found 842149 files belonging to 13 classes.
Training dataset Created.
Saving training dataset...
Training dataset saved.
Creating testing dataset...
Found 842149 files belonging to 13 classes.
Testing dataset Created.
Saving Testing dataset...
Testing dataset saved.
classWeights: {0: 2.7942717429269996, 1: 2.0, 2: 2.8133352088901393, 3: 2.770850651149903, 4: 5.05050505050505, 5: 2.785515320334262, 6: 1.0, 7: 2.8083971073509795, 8: 2.0, 9: 2.811160306416473, 10: 2.764531066417859, 11: 5.051780752715332, 12: 2.848597065945022}


In [20]:
import tensorflow as tf
from keras import callbacks
try:
    model = tf.keras.models.load_model('./ChessModelScaledFinal.keras')
    print("Model loaded.\n")
except:
    print("No saved model found.\n")
    model = False

runTrain = False

if not model or runTrain:
    runTrain = True
    model = tf.keras.Sequential()
    #model = tf.keras.Model()
    #model.add(tf.keras.layers.Input(shape=(None,25,25,1)))
    model.add(tf.keras.layers.Rescaling(1./255))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(13))
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

    earlystopping = callbacks.EarlyStopping(monitor="loss", mode="min", patience=10, restore_best_weights=True, verbose=1)

    #history = model.fit(test_images_ds, epochs=1000, callbacks=[earlystopping], steps_per_epoch=1500, class_weight=classWeights)
    history = model.fit(train_images_ds, epochs=2, callbacks=[earlystopping], class_weight=classWeights)

print("Evaluating Model..")
test_loss, test_acc = model.evaluate(train_images_ds)

print('\nTest accuracy:', test_acc)
if runTrain:
    print("Saving model...")
    model.save('./ChessModelScaledFinal.keras')
    print("Model saved.")


Model loaded.

Evaluating Model..
26318/26318 ━━━━━━━━━━━━━━━━━━━━ 546s 21ms/step - accuracy: 0.1908 - loss: 7.0088

Test accuracy: 0.18999013304710388
